In [8]:
import numpy as np
import pandas as pd

import sklearn.datasets as sd
import sklearn.feature_extraction.text as ft
import sklearn.model_selection as ms
import sklearn.linear_model as lm
import sklearn.naive_bayes as nb
import sklearn.metrics as sm

import nltk.tokenize as tk


In [9]:
data = sd.load_files("20news",encoding="latin1",random_state=7)
data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [10]:
data.filenames.shape
len(data.data)

2968

In [11]:
data.data[0],data.target[0],data.target_names[data.target[0]]

('From: gene@theporch.raider.net (Gene Wright)\nSubject: NASA Special Publications for Voyager Mission?\nOrganization: The MacInteresteds of Nashville, Tn.\nLines: 12\n\nI have two books, both NASA Special Publications, on the Voyager \nMissions. One is titled "Voyages to Jupiter" the other "Voyage to Saturn" \nThese were excellent books put together after the encounters with each \nplanet. \n\nThe question is: Did NASA ever put together a similar book for either the \nUranus encounter or Neptune? If so, what SP number is it and where can it \nbe obtained? If not, why didn\'t they?\n\n--\n  gene@theporch.raider.net (Gene Wright)\ntheporch.raider.net  615/297-7951 The MacInteresteds of Nashville\n',
 4,
 'sci.space')

In [12]:
# 整理输入集与输出集
cv = ft.CountVectorizer()
bow = cv.fit_transform(data.data)
tt = ft.TfidfTransformer()
tfidf = tt.fit_transform(bow)


# 拆分测试集与训练集
train_x,test_x,train_y,test_y=\
    ms.train_test_split(tfidf,data.target,test_size=0.1,stratify=data.target,random_state=7)

# 交叉验证
# model_logic = lm.LogisticRegression()
# score_f1 = ms.cross_val_score(model_logic,train_x,train_y,cv=5,scoring="f1_weighted")
# print(score_f1.mean())


#使用朴素贝叶斯模型
model_NB = nb.MultinomialNB()
score_f1 = ms.cross_val_score(model_NB,train_x,train_y,cv=5,scoring="f1_weighted")
print(score_f1.mean())



#训练模型
model_NB.fit(train_x,train_y)


#测试模型，评估模型
pred_test_y = model_NB.predict(test_x)
print(sm.classification_report(test_y,pred_test_y))



0.9422254301943932
              precision    recall  f1-score   support

           0       1.00      0.83      0.91        58
           1       0.97      1.00      0.98        60
           2       0.98      0.97      0.97        60
           3       0.90      1.00      0.94        60
           4       0.95      0.98      0.97        59

    accuracy                           0.96       297
   macro avg       0.96      0.96      0.96       297
weighted avg       0.96      0.96      0.96       297



# 整理一组测试样本进行模型测试

In [14]:
new_email=["A spectator was hospitalized after being hit by a baseball during the previous game",
          "Wang has been working on asymmetric encryption algorithms",
          "These two wheels are pretty good on the highway",
          "Next year, China will explore Mars."]


# 把样本按照训练时 的方式转换为tfidf矩阵，才可以交给模型训练
bow = cv.transform(new_email)
test_data = tt.transform(bow)

pred_test_data = model_NB.predict(test_data)

# labels=np.array(data.target_names)
labels=pd.Series(data.target_names)
print(labels[pred_test_data])



2    rec.sport.baseball
3             sci.crypt
1       rec.motorcycles
4             sci.space
dtype: object
